In [2]:
from contextlib import contextmanager

In [24]:
from contextlib import ContextDecorator

class mycontext(ContextDecorator):
    def __enter__(self):
        print('Starting', self)
        return self

    def __exit__(self, *exc):
        print('Finishing', self, exc)
        return False

In [29]:
@mycontext()
def handle_event(event, context):
    return {
        'ok': True
    }

In [28]:
handle_event({}, {})

Starting <__main__.mycontext object at 0x7fa4cd3f42e0>
Finishing <__main__.mycontext object at 0x7fa4cd3f42e0> (<class 'ValueError'>, ValueError(), <traceback object at 0x7fa4cd3fad40>)


ValueError: 

In [35]:
import json

In [48]:
def _process_response(payload, code):
    return {
        'Body': json.dumps(payload),
        'StatusCode': code
    }

class response_handling(object):
    def __init__(self, success_code=200):
        print("inside my_decorator.__init__()")
        self.success_code = success_code

    def __call__(self, f):
        print("inside my_decorator.__call__()")
        def wrapped_function(event, context):
            print('event', event)
            try:
                return _process_response(f(event, context), self.success_code) # Prove that function definition has completed            
            except ValueError as ve:
                return _process_response('Something failed', 400)
            except Exception as e:
                return _process_response('Ooops! Sorry! Something bad failed', 500)
        return wrapped_function
        
@response_handling(success_code=201)
def handle_event(event, context):
    print("inside handle_event()", event, context)
#     return {
#         "ok": True
#     }
    raise Exception('i broke')
    

print("Finished decorating handle_event()")

res = handle_event({'something': 7}, {})
print('RESULT', res)

inside my_decorator.__init__()
inside my_decorator.__call__()
Finished decorating handle_event()
event {'something': 7}
inside handle_event() {'something': 7} {}
RESULT {'Body': '"Ooops! Sorry! Something bad failed"', 'StatusCode': 500}
